In [ ]:
% matplotlib inline
from __future__ import print_function

from collections import Counter
import json
import os
import codecs

import lda
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sparse
from sklearn.decomposition import LatentDirichletAllocation

<h2>データを読み込もう</h2>

In [ ]:
with open(os.path.normpath('./dataset/document_word_data.json'), 'r') as f:
    doc_data = json.load(f)

all_doc_index = doc_data.keys()
print('Total Documents: ', len(all_doc_index))

Windowsの方でutf-8のテキストファイルが開けない場合は、こちらを実行してみてください。

In [ ]:
# Windowsの方で上記のコードが失敗する場合
with codecs.open(os.path.normpath('./dataset/document_word_data.json'), 'r', "utf-8") as f:
    doc_data = json.load(f)

all_doc_index = doc_data.keys()
print('Total Documents: ', len(all_doc_index))

この読み込んだjsonデータはこんな感じです。

In [ ]:
', '.join(doc_data['715'])

単語のインデックスを作るために、全ての単語のリストを作ります。

In [ ]:
all_vocab = []
for doc_idx in all_doc_index:
    all_vocab += doc_data[doc_idx]

# 重複を消すためにsetしてlistにする
all_vocab = list(set(all_vocab))
vocab_num = len(all_vocab)
print('Vocablary Number: ', vocab_num)

In [ ]:
all_doc_index_ar = np.array(list(all_doc_index))

train_portion = 0.7
train_num = int(len(all_doc_index_ar) * train_portion)

np.random.shuffle(all_doc_index_ar)
train_doc_index = all_doc_index_ar[:train_num]
test_doc_index = all_doc_index_ar[train_num:]

先にからっぽのスパース行列を定義します。

In [ ]:
A_train = sparse.lil_matrix((len(train_doc_index), len(all_vocab)),
                            dtype=np.int)
A_test = sparse.lil_matrix((len(test_doc_index), len(all_vocab)),
                           dtype=np.int)

ListからNumpyのArrayに直します。

In [ ]:
all_vocab_ar = np.array(all_vocab)
train_doc_index_ar = np.array(train_doc_index)
test_doc_index_ar = np.array(test_doc_index)

スパース行列に成分を入れていきます。

In [ ]:
# 学習用
train_total_elements_num = 0
for i in range(len(train_doc_index)):
    doc_idx = train_doc_index[i]
    row_data = Counter(doc_data[doc_idx])
    
    for word in row_data.keys():
        word_idx = np.where(all_vocab_ar == word)[0][0]
        A_train[i, word_idx] = row_data[word]
        train_total_elements_num += 1
print('Train total elements num :', train_total_elements_num)


# テスト用
test_total_elements_num = 0
for i in range(len(test_doc_index)):
    doc_idx = test_doc_index[i]
    row_data = Counter(doc_data[doc_idx])
    
    for word in row_data.keys():
        word_idx = np.where(all_vocab_ar == word)[0][0]
        A_test[i, word_idx] = row_data[word]
        test_total_elements_num += 1
print('Test total elements num :', test_total_elements_num)

###  CountVectorizerを用いてdoc_dataから簡単に疎行列を作ることもできます

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=lambda a: a, analyzer=lambda a: a)
vectorizer.fit(doc_data[idx] for idx in all_doc_index)
A_train = vectorizer.transform(doc_data[idx] for idx in train_doc_index)
A_test = vectorizer.transform(doc_data[idx] for idx in test_doc_index)

## 実際にLDAを適用してみよう (Scikit-learnを使った例）

In [ ]:
model1 = LatentDirichletAllocation(n_topics=20,
                                   doc_topic_prior=0.001,
                                   topic_word_prior=0.5,
                                   max_iter=5,
                                   learning_method='online',
                                   learning_offset=50.,
                                   random_state=0)

In [ ]:
model1.fit(A_train)

まずトピック x 単語を見てみましょう

In [ ]:
normalize_components = model1.components_ / model1.components_.sum(axis=0)

In [ ]:
# http://scikit-learn.org/stable/auto_examples/applications/
# topics_extraction_with_nmf_lda.html　より
n_top_words = 20
for topic_idx, topic in enumerate(normalize_components):
    print('Topic #%d:' % topic_idx)
    print(' '.join([all_vocab_ar[i] for i in
                    topic.argsort()[:-n_top_words - 1:-1]]))
    print()

文書 x トピック行列側も見てみましょう。

In [ ]:
doc_topic_data = model1.transform(A_train)
doc_topic_data

scikit-learnのLDAはどうやら正規化されていないため、正規化した上で、1つ目の文書がどのトピックから来ている単語が多いのかを見てみましょう。

In [ ]:
normalize_doc_topic_data = \
 doc_topic_data / doc_topic_data.sum(axis=1, keepdims=True)

In [ ]:
for topic_idx, prob in enumerate(normalize_doc_topic_data[0]):
    print('Topic #%d: probality: %f' % (topic_idx, prob))

In [ ]:
loglikelihood = model1.score(A_test)
ppl = model1.perplexity(A_test)
print('対数尤度: ', loglikelihood)
print('Perplexity: ', ppl)

テストデータに当てはめてみましょう。

In [ ]:
test_doc_topic_data = model1.transform(A_test)
normalize_test_doc_topic_data = \
 test_doc_topic_data / test_doc_topic_data.sum(axis=1, keepdims=True)
for topic_idx, prob in enumerate(normalize_test_doc_topic_data[0]):
    print('Topic #%d: probality: %f' % (topic_idx, prob))

<h2> LDAを適用してみよう (ldaパッケージを使った場合)</h2> 

In [ ]:
model2 = lda.LDA(n_topics=20, n_iter=1500, random_state=1, alpha=0.5, eta=0.5)

In [ ]:
model2.fit(A_train)

In [ ]:
topic_word = model2.topic_word_
n_top_words = 20
for topic_idx, topic in enumerate(topic_word):
    print('Topic #%d:' % topic_idx)
    print(' '.join([all_vocab_ar[i] for i in
                    topic.argsort()[:-n_top_words - 1:-1]]))
    print()

今回も精度として対数尤度を見てみましょう

In [ ]:
model2.loglikelihood()

文書 x トピック行列を見てみましょう。

In [ ]:
doc_topic_data2 = model2.transform(A_train)
for topic_idx, prob in enumerate(doc_topic_data2[0]):
    print('Topic #%d: probality: %f' % (topic_idx, prob))

<h2>（参考）ディリクレ分布の挙動</h2>

In [ ]:
alpha = 0.1
K = 6
sampled_probs = np.random.dirichlet([alpha for i in range(K)])
for i, prob in enumerate(sampled_probs):
    print('サイコロ %d面  確率: %.2f'%(i+1, prob))